# IBM Workshop - Natural Language Understanding 

En esta parte del workshop, utilizaremos una instancia de [Watson Natural Language Understanding](https://cloud.ibm.com/catalog/services/natural-language-understanding) para obtener insights de nuestros datos.

Watson Natural Language Understanding es un producto nativo de la nube de IBM que utiliza algoritmos de deep learning para extraer metadata de un texto como pueden ser entidades, palabras clave, categorias, sentimientos, emociones, relaciones y sintaxis

Existe una [API](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python) que estaremos utilizando junto con [Watson Python SDK](https://github.com/watson-developer-cloud/python-sdk) para el análisis de nuestros datos

## Contenido

- [1.0 Configuración - Instalar módulos](#setup)
- [2.0 Prueba de la API de NLU](#test)
- [3.0 Importar datos y creación de un Dataframe en pandas](#pandas)
- [4.0 Limpieza y preparación de datos para el puntaje de la API de NLU](#clean)
- [5.0 Análisis de la respuesta del servicio de NLU ](#analyze)
- [6.0 Obtener el sentimiento por columna](#sentiment-row)
- [7.0 Visualización en una gráfica con matplotlib](#graph)



## 1.0 Configuración - Instalar módulos<a name="setup"></a>

Utilizaremos el [Watson Python SDK](https://github.com/watson-developer-cloud/python-sdk) para accesar a las [NLU APIs](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python) en este notebook.

In [ ]:
!pip install --upgrade numpy==1.16.6
!pip install --upgrade pandas==1.0.5
!pip install --upgrade ibm-watson==4.7.1

### Importante: Reinicia el kernel del Jupiter Notebook 
Reinicia el kernel dando click en el apartado superior llamado `Kernel` y eligiendo la opción `Restart`.

Importar los módulos de python desde el Watson Python SDKs

In [ ]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features,CategoriesOptions,EmotionOptions,KeywordsOptions

### 1.1 Añade las credenciales del servicio de NLU 
Pega la [IAM API Key](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#authentication) y el [Service URL](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#service-endpoint) que obtuviste al crear la instancia de NLU en IBM CLOUD

Reemplaza los asteriscos por tu [IAM API Key](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#authentication).

In [ ]:
IAM_KEY = '*******************'

Reemplaza los asteriscos por tu [NLU Service URL](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#service-endpoint).

In [ ]:
SERVICE_URL = '*******************'

## 2.0 Prueba de la API de NLU <a name="test"></a>
Vamos a correr una pequeña función para verificar que todo esté funcionando correctamente. Usaremos una [pagina web](https://www.ibm.com) para observar como Watson NLU puede extraer categorías cuando se le da un URL. [Este ejemplo](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#categories) está mencionado en la documentación de Watson NLU.

In [ ]:
authenticator = IAMAuthenticator(IAM_KEY)
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2020-08-01',authenticator=authenticator)

natural_language_understanding.set_service_url(SERVICE_URL)

response = natural_language_understanding.analyze(
    url='www.ibm.com',
    features=Features(categories=CategoriesOptions(limit=3))).get_result()

print(json.dumps(response, indent=2))

## 3.0 Importar datos y creación de un Dataframe en pandas <a name="pandas"></a>

Vamos a leer el archivo [cfpciti.csv](https://raw.githubusercontent.com/ibmdevelopermx/Watson-NLU-Workshop/main/Data/cfpbciti.csv) el cual contiene quejas levantadas en el Buró de crédito de consumidores 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('https://raw.githubusercontent.com/ibmdevelopermx/Watson-NLU-Workshop/main/Data/cfpbciti.csv')
df.head(5)

## 4.0 Limpieza y preparación de datos para el puntaje de la API de NLU] <a name="clean"></a>

Estamos interesados en el sentimiento de el cliente hacia varias cosas como el `Product` o el '`Sub-Product`. La columna llamada `Customer complaint narrative` contiene el texto en el cual nos tenemos que enfocar. Vamos a analizarlo.
Las primeras columnas contienen valores 'NaN', así que vamos a ver el valor de la columna 3

In [ ]:
text1 = df.loc[3,"Consumer complaint narrative"]
text1

Ahora vamos a eliminar todas los valores 'NaN' encontrados en la columna de `Consumer complaint narrative` utilizando un método de Pandas llamado [dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

In [ ]:
df2=df['Consumer complaint narrative'].dropna(how = 'all')
df2.head(5)

Vamos a convertir la columna de el dataframe en un string para mandarlo al endpoint del servicio de NLU para evaluación

In [ ]:
df_text = df2.to_string()
df_text

Ahora vamos a mandar estos datos a Watson para obtener [palabras clave (keywords)](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#keywords), [sentimientos (sentiment)](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#sentiment) y [emociones (emotions)](https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#emotion) .

In [ ]:
response = natural_language_understanding.analyze(
    text = df_text,
    features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=5))).get_result()

print(json.dumps(response, indent=2))

En el resultado obtenido se puede observar que se tomaron los campos 'XX/XX' de las fechas que están seleccionadas. Como esto ocurre frecuentemente en este dataset, la respuesta de el servicio marca este valor como algo de alta relevancia. Vamos a eliminar los valores 'XX' para obtener una respuesta mas clara de Watson.

In [ ]:
df2 =df2.replace(regex=['X+'],value='')

In [ ]:
df_text = df2.to_string()
df_text

Ya con esos valores eliminados, vamos a volver a mandar la petición a el servicio.

In [ ]:
response = natural_language_understanding.analyze(
    text = df_text,
    features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=5))).get_result()

print(json.dumps(response, indent=2))

El servicio arrojó información que podemos utilizar. Se puede observar en la respuesta de el servicio que hay un límite de 50,000 caracteres. Vamos a corregir eso mas adelante, por ahora vamos a analizar los datos obtenidos

## 5.0 Análisis de la respuesta del servicio de NLU  <a name=analyze></a>

Vamos a crear un dataframe con la respuesta de la API. Primero vamos a observar una parte de la respuesta en formato json asociada con la llave `keywords`.

In [ ]:
respj = json.dumps(response['keywords'])
respj

Se ve genial! Ahora vamos a crear un dataframe con ese json utilizando el método [read_json()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html).

In [ ]:
json_df = pd.read_json(respj)
json_df.head()

Esto funciona, pero la columna `Sentiment` y `emotion` estan compuestas de un json que contiene múltiples valores en un [diccionario de Python](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).
Podemos utilizar la función [json_normalize()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) para crear un dataframe que separa las columnas `sentiment`y `emotion`. Vamos a eliminar algúnas columnas de nuestro dataframe para enfocarnos solamente en `sentiment.score` y en `emotion`

In [ ]:
norm_df = pd.json_normalize(response['keywords'])
norm_df.drop('relevance',inplace = True, axis = 1)
norm_df.drop('count',inplace = True, axis = 1)
norm_df.drop('sentiment.label',inplace = True, axis = 1)
norm_df.head()

Esta exploración de los datos nos otorga herramientas para trabajar. Vamos a continuar el análisis del texto en las siguientes secciones.

## 6.0 Obtener el sentimiento por columna <a name="sentiment-row"></a>

Ahora, obtengamos información sobre sentimientos y emociones por fila, para proporcionar más granularidad.
La cantidad de llamadas a la API que puede realizar a Watson NLU [contiene un limite y depende de tu plan de servicio](https://cloud.ibm.com/catalog/services/natural-language-understanding) (En nuestro caso es el plan lite), asi que para limitar estas llamadas a la API de NLU vamos a empezar con solo 50 filas definiendo el valor `num_rows` en 50.

In [ ]:
num_rows = 50

In [ ]:
df_rows = df.head(num_rows)
df_rows = df_rows.dropna(subset=['Consumer complaint narrative'],how = 'any')
df_rows =df_rows.replace(regex=['X+'],value='')
df_rows.head()

Notamos que cuando eliminamos las columnas que contenian valores `NaN` en `Customer complaint narrative` los indices de las tablas ya no eran secuenciales. Vamos a utilizar el método [reset_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) para solucionarlo

In [ ]:
df_rows.reset_index(drop=True, inplace=True)

Existen muchas maneras de iterar en las filas de un dataframe, nosotros vamos a utilizar el método [iterrows()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html)

Primero, tenemos una fecha para esas entradas, vamos a colocarlas en un formato [Pandas datetime](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html). Podemos utilizar esto mas tarde para hacer graficas en series de tiempo.

In [ ]:
for index, row in df_rows.iterrows():
    df_rows.loc[index,'Date received'] = datetime.strptime(row['Date received'], "%m/%d/%y")

Ahora, vamos a buscar algo que podamos usar con Watson NLU para el análisis de sentimiento de la retroalimentación del cliente.

In [ ]:
df_rows.head()

In [ ]:
print (df_rows['Consumer complaint narrative'][0])

Esto se ve como algo que nos interesa. Ahora vamos a crear una lista para guardar estas `respuestas`, llamar a el servicio de NLU con los datos y llenar esta lista de respuestas. Haremos lo mismo con una lista llamada `normalize` que podemos utilizar junto con el método [json_normalize()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html).

In [ ]:
responses = []
normalize = []
for index, row in df_rows.iterrows():
    
    response = natural_language_understanding.analyze(
    text = row['Consumer complaint narrative'],
    features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=1))).get_result()
    normalize.append(pd.json_normalize(response['keywords']))
    responses.append(response)
    

In [ ]:
normalize

In [ ]:
responses

Añadimos la lista de `responses` y la de `normalize`a un df_rows dataframe. Podemos seguir usando estas nuevas características de datos, pero más comúnmente derivaremos nuevos marcos de datos para nuestros experimentos y cambiaremos esos nuevos marcos de datos en su lugar.

In [ ]:
df_rows['response'] = responses
df_rows.head()

In [ ]:
df_rows['normalized'] = normalize
df_rows.head()

Vamos a crear un nuevo dataframe en donde podamos obtener la columna para la `emocion` `anger` y despues ordenarla por el rating mas alto de `anger`.

In [ ]:
test_df = df_rows

In [ ]:
for index, row in test_df.iterrows():
    test_df.loc[index,"anger"] = test_df.iloc[index]['response']['keywords'][0]['emotion']['anger']
    test_df.loc[index,"sentiment.score"] = test_df.iloc[index]['response']['keywords'][0]['sentiment']['score']

In [ ]:
test_df.head()

Vamos a ver primero las entradas que tienen mayor rango de `anger`

In [ ]:
sorted_df = test_df.sort_values(by='anger', ascending=False)
sorted_df.head()

Ahora, vamos a observar los valores de la columna `Consumer complaint narrative` que causan más enojo (La que está a la cabeza de la lista ordenada)

In [ ]:
sorted_df.iloc[0]['Consumer complaint narrative']

Podemos hacer lo mismo para otros valores que contengan un rango alto en la lista de enojo.

In [ ]:
sorted_df.iloc[1]['Consumer complaint narrative']

Vamos a observar los valores que contienen el mas alto sentimiento de negatividad. Para esto vamos a ordenar de manera ascendente.

In [ ]:
sorted_df = test_df.sort_values(by='sentiment.score', ascending=True)
sorted_df.head()

In [ ]:
sorted_df.iloc[0]['Consumer complaint narrative']

Bueno, no es sorprendente que la entrada con el sentimiento mas negativo sea la misma que la que tienen mas rating en `anger`

## 7.0 Visualización en una gráfica con matplotlib <a name="graph"></a>

Vamos a crear algúnas gráficas  utilizando la librería [matplotlib](https://matplotlib.org). Si gustas puedes explorar mas detalles acerca de las magic functions](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#magics-explained) que se utilizan en los Jupyter notebook que puedes ver con el comando `%matplotlib inline`.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

### 7.1 Gráfica en series de tiempo

Podemos observar si hay algo interesante cuando ponemos los datos contra el tiempo.

In [ ]:
sorted_df.plot(kind='line',x='Date received',y='anger',color='red')

In [ ]:
sorted_df.plot(kind='line',x='Date received',y='sentiment.score',color='blue')

Now we can plot both `sentiment.score` and `anger` against time to look for correlations.
Ahora vamos a poner el `sentiment.score` y el `anger` contra el tiempo para buscar alguna correlación. 

In [ ]:
sorted_df.plot(kind='line',x='Date received',y=['sentiment.score','anger'])

### 7.2 Gráficas de Barra

Podemos sumar el número de veces en las que aparece la columna `Product` o `Sub-product` utilizando la funcion de [Python collections library Counter](https://docs.python.org/2/library/collections.html#collections.Counter)

In [ ]:
from collections import Counter

Después, camos a crear una gráfica de barras para ver a que `Product` se refieren más en las quejas de los clientes.

In [ ]:
bar_hist = Counter(sorted_df['Product'].replace('\n', ''))

counts = bar_hist.values()
letters = bar_hist.keys()

# graph data
bar_x_locations = np.arange(len(counts))
plt.bar(bar_x_locations, counts, align = 'center')
plt.xticks(bar_x_locations, letters, rotation=90)
plt.grid()
plt.show()

Podemos hacer lo mismo también para `Sub-product`.

In [ ]:
bar_hist = Counter(sorted_df['Sub-product'].replace('\n', ''))

counts = bar_hist.values()
letters = bar_hist.keys()

# graph data
bar_x_locations = np.arange(len(counts))
plt.bar(bar_x_locations, counts, align = 'center')
plt.xticks(bar_x_locations, letters, rotation=90)
plt.grid()
plt.show()

### 7.3 Gráfico de dispersión

Finalmente, vamos a utilizar matplotlib y numpy para generar un gráfico de dispersión en 3D

In [ ]:
import mpl_toolkits.mplot3d.axes3d as axes3d


Xuniques, X = np.unique(sorted_df['Sub-product'], return_inverse=True)
Yuniques, Y = np.unique(sorted_df['Product'], return_inverse=True)
Z= sorted_df['anger']
fig = plt.figure(figsize= [15,8])
ax = fig.add_subplot(1, 1, 1, projection='3d',autoscale_on=True)
ax.scatter(X, Y, Z, s=10, c='b')
ax.set(xticks=range(len(Xuniques)), xticklabels=Xuniques,
       yticks=range(len(Yuniques)), yticklabels=Yuniques)
plt.xticks(rotation=90)
plt.show()


Y listo! No olvides visitar el [Github de IBM Developer en español](https://github.com/ibmdevelopermx) para encontrar mas temas acerca de Watson y sus API's